<a href="https://colab.research.google.com/github/ryogrid/ryogridJupyterNotebooks/blob/master/llambaindex_with_bert_fail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/llamaindex-with-bert

/content/drive/MyDrive/llamaindex-with-bert


In [3]:
!cat requirements.txt

llama-index
langchain
transformers
sentence-transformers
sentencepiece

In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.5/493.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 118.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11

In [5]:
# 日本語対応パッケージのインストール
!pip install -q fugashi[unidic-lite]
!pip install -q ipadic

# Huggingface Datasetsのインストール
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.9/599.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 54.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.7 MB/s eta 0:00:00


In [6]:
from llama_index import LLMPredictor
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

In [ ]:
!pwd
!ls

/content/drive/MyDrive/gpt-index-trial
requirements.txt


In [7]:
DEFAULT_PROMPT = """
文脈情報は以下です。
---
{context_str}
---
事前知識ではなく、文脈情報を参考に質問に答えてください。：{query_str}
"""

REFINE_PROMPT = """
質問は以下です。：{query_str}
すでに答えの候補があります。：{existing_answer}
必要な場合のみ、以下の文脈情報を使ってこの答えを改良することができます。
---
{context_msg}
---
この文脈情報により、元の答えを改良して質問に答えてください。
文脈情報が有用でない場合は元の答えをそのまま返してください。
"""

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
qa_model = AutoModelForCausalLM.from_pretrained("ryogrid/bert-base-japanese-whole-word-masking-qa-tuned", is_decoder=True)

if torch.cuda.is_available():
    qa_model = qa_model.to("cuda")


Some weights of the model checkpoint at ryogrid/bert-base-japanese-whole-word-masking-qa-tuned were not used when initializing BertLMHeadModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at ryogrid/bert-base-japanese-whole-word-masking-qa-tuned and are newly initialized: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.de

In [11]:
def generate(prompt):

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    n = len(token_ids[0])

    with torch.no_grad():
        output_ids = qa_model.generate(
            token_ids.to(qa_model.device),
            max_length=n+100,
            min_length=n+2,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids.tolist()[0][n:])
    return output.replace("</s>", "")

In [12]:
class CustomLLM(LLM):
        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return generate(prompt)
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name":"custom"}
llm = CustomLLM()
llm_predictor = LLMPredictor(llm=llm)
# llm_predictor = LLMPredictor(llm)

In [13]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

# load in HF embedding model from langchain
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="ryogrid/bert-base-japanese-whole-word-masking-qa-tuned"))
embed_model._langchain_embedding.client.max_seq_length = 256

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/ryogrid_bert-base-japanese-whole-word-masking-qa-tuned were not used when initializing BertModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/ryogrid_bert-base-japanese-whole-word-masking-qa-tuned and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to 

In [14]:
import urllib
import json
url = "https://ja.wikipedia.org/wiki/%E3%81%BC%E3%81%A3%E3%81%A1%E3%83%BB%E3%81%96%E3%83%BB%E3%82%8D%E3%81%A3%E3%81%8F!?action=cirrusdump"
with urllib.request.urlopen(url) as f:
    data = f.read()
text = json.loads(data)[0]["_source"]["text"]

In [ ]:
# OpenAIのAPIは呼び出さないがapi keyが設定されていないとインデックス作成時にエラーになる
# ため、空文字を設定している
! export OPENAI_API_KEY=""

In [15]:
#from llama_index import GPTVectorStoreIndex
from llama_index import LLMPredictor, ServiceContext, GPTSimpleKeywordTableIndex, VectorStoreIndex
from llama_index.readers.schema.base import Document
from llama_index.indices.list.base import ListRetrieverMode
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.indices.service_context import set_global_service_context
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.query.response_synthesis import ResponseSynthesizer
from llama_index.indices.response.type import ResponseMode

documents = []
for i in range(0, len(text), 200):
    documents.append(Document(text[i:i+200]))
    if i != 0:
        documents.append(Document(text[i-100:i+100]))

#llm_predictor   = LLMPredictor(llm=llm)
llama_debug_handler = LlamaDebugHandler()
callback_manager = CallbackManager([llama_debug_handler])
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, callback_manager=callback_manager, embed_model=embed_model)
#set_global_service_context(service_context=service_context)
#service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# インデックスの作成
#index = GPTVectorStoreIndex.from_documents(documents, llm_predictor=llm_predictor, embed_model=embed_model)
#index = GPTSimpleKeywordTableIndex.from_documents(documents, service_context=service_context, llm_predictor=llm_predictor, embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents, service_context=service_context, llm_predictor=llm_predictor, embed_model=embed_model)

**********
Trace: index_construction
    |_node_parsing ->  0.093515 seconds
      |_chunking ->  0.001135 seconds
      |_chunking ->  0.000384 seconds
      |_chunking ->  0.000298 seconds
      |_chunking ->  0.000551 seconds
      |_chunking ->  0.000309 seconds
      |_chunking ->  0.000363 seconds
      |_chunking ->  0.000272 seconds
      |_chunking ->  0.00029 seconds
      |_chunking ->  0.000288 seconds
      |_chunking ->  0.000367 seconds
      |_chunking ->  0.000277 seconds
      |_chunking ->  0.000299 seconds
      |_chunking ->  0.000281 seconds
      |_chunking ->  0.000265 seconds
      |_chunking ->  0.00027 seconds
      |_chunking ->  0.000406 seconds
      |_chunking ->  0.000285 seconds
      |_chunking ->  0.000292 seconds
      |_chunking ->  0.000308 seconds
      |_chunking ->  0.000297 seconds
      |_chunking ->  0.000269 seconds
      |_chunking ->  0.000287 seconds
      |_chunking ->  0.000281 seconds
      |_chunking ->  0.000362 seconds
      |_chunk

In [16]:
len(documents)

253

In [17]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompts import RefinePrompt, QuestionAnswerPrompt
refine_prompt = RefinePrompt(REFINE_PROMPT)
default_prompt = QuestionAnswerPrompt(DEFAULT_PROMPT)

In [18]:
#query_engine = index.as_query_engine()

'''
retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=2,
)
'''

retriever = index.as_retriever(retriever_mode='embedding',  similarity_top_k=3)
response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.REFINE ,service_context=service_context, text_qa_template=default_prompt, refine_template=refine_prompt, callback_manager=callback_manager, verbose=True)
query_engine = RetrieverQueryEngine(retriever=retriever, response_synthesizer=response_synthesizer, callback_manager=callback_manager)
'''
query_engine = index.as_query_engine(
    retriever_mode=ListRetrieverMode.EMBEDDING
    , similarity_top_k=3
)
'''

response = query_engine.query(
    "虹夏ちゃんのお姉さんの仕事は？"
)
'''
response = index.query(
    "虹夏ちゃんのお姉さんの仕事は？", 
    mode="embedding", 
    verbose=True, 
    embed_model=embed_model,
    text_qa_template=default_prompt,
    refine_template=refine_prompt,
    similarity_top_k=2
)
'''
response.response

**********
Trace: query
    |_query ->  6.977576 seconds
      |_retrieve ->  0.090791 seconds
        |_embedding ->  0.032278 seconds
      |_synthesize ->  6.886603 seconds
        |_llm ->  3.015916 seconds
        |_llm ->  2.954407 seconds
        |_llm ->  0.912544 seconds
**********


'##もん 途リエンリエンリエンもんもんもんもんリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンもんもんリエンリエンリエンリエンリエンもん 日生リエンリエンもん 日生リエンリエンリエンリエンリエンリエンリエンリエンリエンもんもんもんリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエン 途 途 途 日生洋 途 途 抱 日生 日生 途 途 日生 日生 途 途 日生 途 途 日生 日生 途 途 途 途 日生 途 途 途 途 途'

In [19]:
response = query_engine.query(
    "後藤ひとりがギターに熱中するようになった理由になった出来事は？", 
)
'''
response = index.query(
    "後藤ひとりがギターに熱中するようになった理由になった出来事は？", 
    mode="embedding", 
    verbose=True, 
    embed_model=embed_model,
    text_qa_template=default_prompt,
    refine_template=refine_prompt,
    similarity_top_k=2
)
'''

response.response

**********
Trace: query
    |_query ->  4.373507 seconds
      |_retrieve ->  0.147477 seconds
        |_embedding ->  0.034288 seconds
      |_synthesize ->  4.225828 seconds
        |_llm ->  1.384398 seconds
        |_llm ->  1.514366 seconds
        |_llm ->  1.322366 seconds
**********


'##もんもんもんもんもんもんもんもんもんもんもんリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンもんもんもんもんもんもんもんもんリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンもんもんリエンリエンリエンリエンリエンもんもんもんもんもんもんもんもんリエンリエンリエンリエンリエンリエンもんもんもんリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンもんもんもんもんもんもんもんもんもんもんもんもんリエンリエンリエンもん'

In [20]:
response = query_engine.query(
    "ギターヒーローの正体は誰？", 
)
response.response


**********
Trace: query
    |_query ->  2.813059 seconds
      |_retrieve ->  0.100819 seconds
        |_embedding ->  0.021265 seconds
      |_synthesize ->  2.712087 seconds
        |_llm ->  0.889854 seconds
        |_llm ->  0.893502 seconds
        |_llm ->  0.925182 seconds
**********


'紛リエン 三国 三国 三国 三国 ぎふ ><リエンリエンリエンリエンリエンリエンもん 日生リエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンもんもんリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンリエンもんもんもんもんリエンリエンリエンもんもんもんもんもんもんもんもんリエンリエンリエンもんもんもんもんもんもんもんもんもんもんもん 三国陀 ><リエン 三国 日生 途 途 日生リエン 途 日生 日生リエン 途 日生 日生 途リエン 途 日生 日生リエン'

In [ ]:
response = query_engine.query(
    "喜多郁代の両親の仕事は？", 
)
response.response

**********
Trace: query
    |_query ->  0.880911 seconds
      |_retrieve ->  0.084083 seconds
        |_embedding ->  0.021573 seconds
      |_synthesize ->  0.796681 seconds
        |_llm ->  0.210591 seconds
        |_llm ->  0.226277 seconds
        |_llm ->  0.130148 seconds
        |_llm ->  0.097139 seconds
        |_llm ->  0.127703 seconds
**********


'病院の経営'

In [ ]:
response = query_engine.query(
    "後藤ひとりの配信している時の名は何ですか？", 
)
response.response

**********
Trace: query
    |_query ->  0.800553 seconds
      |_retrieve ->  0.085335 seconds
        |_embedding ->  0.020418 seconds
      |_synthesize ->  0.715048 seconds
        |_llm ->  0.264401 seconds
        |_llm ->  0.268415 seconds
        |_llm ->  0.178701 seconds
**********


'ひとり(ごとう)'